# Limpieza de datos

Limpieza de data set para Visualización de la información.

In [126]:
import pandas as pd
import geopandas as gpd
import re

In [206]:
# Dataset de Kaggle
data1 = pd.read_csv('data/15_to_30_victim_count.csv')
data2 = pd.read_csv('data/5_to_14_victim_count.csv')
data3 = pd.read_csv('data/Highest_victim_count.csv')
data4 = pd.read_csv('data/Lessthan_5_victim_count.csv')

#Mapa en geojson
mapa = gpd.read_file('https://raw.githubusercontent.com/simonepri/geo-maps/master/previews/countries-coastline.geo.json')

data = pd.concat([data1,data2,data3,data4])
data

# Dicccionario para ISO A3 code para países
country_dict = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv',
                           encoding = 'UTF-8')

## Limpieza de país

Genero pais 1, 2 y 3 para campos con más de un país. Se eliminan los que tienen más de 3 países para simplicidad del análisis

In [172]:
# Generar pais 1-2-3
pais = data["Country"].str.split("\r\n", n = 2, expand = True)

data['pais_1'] = pais[0]
data['pais_2'] = pais[1]
data['pais_3'] = pais[2]

cols = ['pais_1','pais_2','pais_3']

for col in cols:
    data[col] = (data[col].str.replace('United States of America','United States ').replace(' \(suspected\)',"", regex=True)
    .replace('Soviet Union','Russia').replace("United States Mexico","United States").replace(' \(claimed\)',"", regex=True).replace(" \(alleged\)'',", regex=True).replace("East Germany","Germany")
    .replace("West Germany","Germany", regex=True).replace('Austria-Hungary',"Austria").replace("^East Germany","Germany",  regex=True).replace('German Empire','Germany')
    .replace('Allied-occupied Germany',"Germany").replace('Kingdom of Romania',"Romania")
    )


# Limpieza específica de pais 2 y 3
data.pais_2 = (data.pais_2.replace(' \(alleged\)',"", regex=True).replace("\r\n.*","", regex=True).replace('\n.*',"",regex=True)
)
data.pais_3 = (data.pais_3.replace(' \(alleged\)',"", regex=True).replace("\r\n.*","", regex=True).replace('\n.*',"",regex=True)
)


## Limpieza campo año

Limpio años y divido en año de inicio y año de termino.

In [209]:
# limpio datos 
data['Years active'] = (data['Years active'].replace('late ',"", regex=True).replace("s","", regex=True).replace(' and earlier',"", regex=True)
.replace("\?","", regex=True).replace('c.',"", regex=True).replace('30 June ',"", regex=True).replace('preent',"2020", regex=True)
.replace('23 July ',"", regex=True)
)

# divido en año inicio y termino

años  = data["Years active"].str.split(" to ", n = 1, expand = True)
data['año_inicio'] = años[0]
data['año_termino'] = años[1]

0     1924.0
1     2003.0
2     2013.0
3     2001.0
4     2000.0
       ...  
24    1940.0
25       NaN
26    1932.0
27    1996.0
28       NaN
Name: año_termino, Length: 305, dtype: float64